In [1]:
# GENETIC ALGORITHM for solving linear regression:
# we want to solve: y = a(x1) + b(x2) + c(x3) + d --> find best value for paramter a,b,c,d
#
# Source:https://medium.com/the-andela-way/on-genetic-algorithms-and-their-application-in-solving-regression-problems-4e37ac1115d5
# https://medium.com/the-andela-way/on-genetic-algorithms-and-their-application-in-solving-regression-problems-4e37ac1115d5
# # rewritten by D.Adytia (didit.adytia@gmail.com) ver. 2021-04-20

from random import random, sample, choice
from math import floor
from tqdm import tqdm                  # instantly make your loops show a smart progress meter
from numpy import array, dot, mean
from numpy.linalg import pinv
from sys import exit

In [15]:
def generate_data():
    """
    We will generate data with a clear pattern.
    This ensures we have an idea of the desired result.
    This is only for demonstration purposes, real data is needed in practice.
    """
    coeff = [0.4, -0.3, 0.2, -0.1]
    x = [[random() for j in range(len(coeff))] for i in range(1000)]   # generate random data set with range of 1000
    y = [dot(i, coeff) for i in x]
    return array(x), array(y)


In [3]:
def multiple_linear_regression(inputs, outputs):
    """
    Get the best expected outcome.
    This is expected to equal the coefficients in generate_data().
    """
    X, Y = array(inputs), array(outputs)
    X_t, Y_t = X.transpose(), Y.transpose()
    coeff = dot((pinv((dot(X_t, X)))), (dot(X_t, Y)))
    Y_p = dot(X, coeff)
    Y_mean = mean(Y)
    SST = array([(i - Y_mean) ** 2 for i in Y]).sum()         # SST: the total error in a model, it is the sum of all deviations squared
    SSR = array([(i - j) ** 2 for i, j in zip(Y, Y_p)]).sum() # SSR: a measure of the explained variation in SST.
    COD = (1 - (SSR / SST)) * 100.0                           # COD: stands for ‘coefficient of determination’ which is basically a measure of how good a model is.
    av_error = (SSR / len(Y))
    return {'COD': COD, 'coeff': coeff, 'error': av_error}    # error: the average error, is an average of all deviations from expected values

In [4]:
def check_termination_condition(best_individual):
    """
    Check if the current_best_individual is better of equal to the expected.
    """
    if ((best_individual['COD'] >= 99.5)
            or (generation_count == max_generations)):
        return True
    else:
        return False

In [5]:
def create_individual(individual_size):
    """
    Create an individual.
    """
    return [random() for i in range(individual_size)]   # To create an initial individual, we will use random assigning of variables.


def create_population(individual_size, population_size):
    """
    Create an initial population.
    """
    return [create_individual(individual_size) for i in range(population_size)]


def get_fitness(individual, inputs):
    """
    Calculate the fitness of an individual.
    Return the Coefficient of Determination, average error and weight.
    We use the error to get the best individual.
    """
    predicted_outputs = dot(array(inputs), array(individual))
    output_mean = mean(outputs)
    SST = array(
        [(i - output_mean) ** 2 for i in outputs]).sum()
    SSR = array(
        [(i - j) ** 2 for i, j in zip(outputs, predicted_outputs)]).sum()
    COD = (1 - (SSR / SST)) * 100.0
    av_error = (SSR / len(outputs))
    return {'COD': COD, 'error': av_error, 'coeff': individual}


def evaluate_population(population):
    """
    Evaluate a population of individuals and return the best among them.
    """
    fitness_list = [get_fitness(individual, inputs)
                    for individual in tqdm(population)]
    error_list = sorted(fitness_list, key=lambda i: i['error'])
    best_individuals = error_list[: selection_size]
    best_individuals_stash.append(best_individuals[0]['coeff'])
    print('Error: ', best_individuals[0]['error'],
          'COD: ', best_individuals[0]['COD'])
    return best_individuals


def crossover(parent_1, parent_2):
    """
    Return offspring given two parents.
    Unlike real scenarios, genes in the chromosomes aren't necessarily linked.
    """
    child = {}
    loci = [i for i in range(0, individual_size)]
    loci_1 = sample(loci, floor(0.5*(individual_size)))
    loci_2 = [i for i in loci if i not in loci_1]
    chromosome_1 = [[i, parent_1['coeff'][i]] for i in loci_1]
    chromosome_2 = [[i, parent_2['coeff'][i]] for i in loci_2]
    child.update({key: value for (key, value) in chromosome_1})
    child.update({key: value for (key, value) in chromosome_2})
    return [child[i] for i in loci]


def mutate(individual):
    """
    Mutate an individual.
    The gene transform decides whether we'll add or deduct a random value.
    """
    loci = [i for i in range(0, individual_size)]
    no_of_genes_mutated = floor(probability_of_gene_mutating*individual_size)
    loci_to_mutate = sample(loci, no_of_genes_mutated)
    for locus in loci_to_mutate:
        gene_transform = choice([-1, 1])
        change = gene_transform*random()
        individual[locus] = individual[locus] + change
    return individual


def get_new_generation(selected_individuals):
    """
    Given selected individuals, create a new population by mating them.
    Here we also apply variation operations like mutation and crossover.
    """
    parent_pairs = [sample(selected_individuals, 2)
                    for i in range(population_size)]
    offspring = [crossover(pair[0], pair[1]) for pair in parent_pairs]
    offspring_indices = [i for i in range(population_size)]
    offspring_to_mutate = sample(
        offspring_indices,
        floor(probability_of_individual_mutating*population_size)
    )
    mutated_offspring = [[i, mutate(offspring[i])]
                         for i in offspring_to_mutate]
    for child in mutated_offspring:
        offspring[child[0]] = child[1]
    return offspring

In [20]:
# MAIN CODE
# Initialization
inputs, outputs = generate_data()
print(inputs.shape)
print(outputs)

(1000, 4)
[ 4.53338204e-02  9.55432232e-02  1.48594376e-01  3.26552161e-01
  4.78654690e-02  3.34553929e-01  9.46616847e-02  9.27947913e-02
 -2.39964232e-02  2.07128745e-01 -2.31925009e-01  2.41423073e-01
 -2.95257047e-02  5.82288782e-03  2.04397491e-01  2.13304002e-01
  5.96764826e-02  2.98452556e-01  9.26594512e-02  2.39317721e-01
 -1.20830537e-01  1.89683135e-01 -1.69393722e-01  2.36097533e-01
  2.60245807e-01  5.85256541e-02  1.43807328e-01  1.45468362e-01
  7.73631866e-03  5.27880836e-02  3.60643059e-01  3.22371040e-01
  1.59471145e-01  8.17922653e-02  1.86024583e-01  3.10917540e-01
  2.56409992e-01  9.05280430e-02  1.07569623e-01  3.01570840e-01
  1.93499023e-01  1.80983288e-01  1.18718920e-01  9.09142706e-02
  9.99873006e-02 -2.11846376e-02 -1.95293122e-02 -2.71071301e-01
  3.27438512e-01  1.48913122e-02  2.52361682e-01 -1.26819762e-01
  2.27086396e-01  1.72222168e-01 -1.75918987e-02  8.04053026e-02
  7.65095638e-02 -1.06764029e-02 -1.11595761e-01 -2.08870012e-01
  1.78931637e-0

In [7]:
individual_size = len(inputs[0])
population_size = 1000
selection_size  = floor(0.1*population_size)
max_generations = 50
probability_of_individual_mutating = 0.1
probability_of_gene_mutating = 0.25
best_possible = multiple_linear_regression(inputs, outputs)
best_individuals_stash = [create_individual(individual_size)]
initial_population = create_population(individual_size, 1000)
current_population = initial_population
termination = False
generation_count = 0
while termination is False:
    current_best_individual = get_fitness(best_individuals_stash[-1], inputs)
    print('Generation: ', generation_count)
    best_individuals   = evaluate_population(current_population)
    current_population = get_new_generation(best_individuals)
    termination        = check_termination_condition(current_best_individual)
    generation_count  += 1
else:
    print(get_fitness(best_individuals_stash[-1], inputs))

Generation:  0


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1872.90it/s]


Error:  0.030112516572258087 COD:  -19.657962318447765
Generation:  1


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1850.41it/s]


Error:  0.018843949208723322 COD:  25.119889633353452
Generation:  2


...

Error:  0.0001455454163269207 COD:  99.42164687894223
Generation:  12


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1836.27it/s]


Error:  0.00010480329389433024 COD:  99.5835436549594
Generation:  13


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1870.97it/s]

Error:  0.00010480329389433024 COD:  99.5835436549594
{'COD': 99.5835436549594, 'error': 0.00010480329389433024, 'coeff': [0.3766143783628948, -0.2767117222498273, 0.19860916678567064, -0.0922892071773368]}


In [8]:
len(outputs)

1000